# Plus proches voisins en grande dimension

La méthodes des [plus proches voisins](https://fr.wikipedia.org/wiki/Recherche_des_plus_proches_voisins) est un algorithme assez simple. Que se passe-t-il quand la dimension de l'espace des features augmente ? Comment y remédier ? Le profiling [memory_profiler](https://pypi.python.org/pypi/memory_profiler) ou [cprofile](https://docs.python.org/3.7/library/profile.html#module-cProfile) sont des outils utiles pour savoir où le temps est perdu. 

## Q1 : k-nn : mesurer la performance

In [1]:
from sklearn.datasets import make_classification

datax, datay = make_classification(
    10000, n_features=10, n_classes=3, n_clusters_per_class=2, n_informative=8
)
datax[:3]

array([[-0.85382076,  0.22046675,  1.24910001,  2.94596312,  0.66829759,
        -1.20552856, -1.72023578, -1.84674932, -0.26846378,  0.20075415],
       [-1.59306412,  1.88866079, -0.76923866, -2.32519462, -2.94535057,
        -1.47877141, -2.2276281 ,  0.02957725,  1.85438519,  0.55194846],
       [ 5.58758523,  2.80964683,  0.32608346,  4.12806316, -1.3248342 ,
         1.06996005,  2.31117628,  3.99525892, -1.47020431, -4.13399841]])

In [2]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(5, algorithm="brute")
model.fit(datax, datay)

KNeighborsClassifier(algorithm='brute')

In [3]:
model.predict(datax)

array([0, 1, 0, ..., 2, 0, 1])

In [4]:
%timeit model.predict(datax)

271 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
import numpy
import os

path = os.path.normpath(os.path.join(numpy.__file__, "..", ".."))
print(path)

C:\Python3105_x64\lib\site-packages


In [6]:
import cProfile
import pstats
from io import StringIO

pr = cProfile.Profile()
pr.enable()
model.predict(datax)
pr.disable()
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")
ps.print_stats()
res = s.getvalue().replace(path, "").replace("\\", "/").replace(" /", " ")
print("\n".join(res.split("\n")[:50]))

         445 function calls in 0.287 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.287    0.143 IPython/core/interactiveshell.py:3397(run_code)
        2    0.000    0.000    0.287    0.143 {built-in method builtins.exec}
        1    0.000    0.000    0.287    0.287 C:/Users/xavie/AppData/Local/Temp/ipykernel_22912/2929265561.py:1(<module>)
        1    0.000    0.000    0.287    0.287 C:/xavierdupre/__home_/github_fork/scikit-learn/sklearn/neighbors/_classification.py:230(predict)
        1    0.001    0.001    0.286    0.286 C:/xavierdupre/__home_/github_fork/scikit-learn/sklearn/neighbors/_classification.py:291(predict_proba)
        1    0.000    0.000    0.285    0.285 C:/xavierdupre/__home_/github_fork/scikit-learn/sklearn/neighbors/_base.py:730(kneighbors)
        1    0.000    0.000    0.283    0.283 C:/xavierdupre/__home_/github_fork/scikit-learn/sklearn/metrics/_pairwise_dista

Etudier l'évolution du temps de prédiction en fonction du nombre d'observations, de la dimension, du nombre de classes ? Qu'en déduisez-vous ? Le code sur GitHub :

* [predict](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/classification.py#L129)
* [kneighbors](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a805efbe4bb06516670a9b8c690992bd7/sklearn/neighbors/base.py#L273)
* [pairwise_distance](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/pairwise.py#L1141)

## Q2 : k-nn avec sparse features

On recommence cette mesure de temps mais en créant des jeux de données [sparses](https://fr.wikipedia.org/wiki/Matrice_creuse).

## Q3 : Imaginez une façon d'aller plus vite ?

Aller plus vite veut parfois dire perdre un peu en performance et dans notre cas, on veut accélérer la prédiction.